In [24]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

load_dotenv()

True

In [25]:
model = ChatOpenAI()

In [26]:
# define the state
class ReviewState(TypedDict):

    item: str
    review: str
    evaluation: str

In [27]:
def generate_review(state: ReviewState) -> ReviewState:
    item = state["item"]
    prompt = f"Generate a random negative review for {item}"
    review = model.invoke(prompt).content
    state["review"] = review
    return state

In [28]:
def evaluate_review(state: ReviewState) -> ReviewState:
    review = state["review"]
    prompt = f"Evaluate the below review\n{review}\n If the review is positive thank the customer but if it is negative then give the customer other options."
    evaluation_string = model.invoke(prompt).content
    state["evaluation"] = evaluation_string
    return state

In [29]:
# define the graph
graph = StateGraph(ReviewState)
# add nodes
graph.add_node("Review_Generation", generate_review)
graph.add_node("Review_Evaluation", evaluate_review)
# add edges
graph.add_edge(START, "Review_Generation")
graph.add_edge("Review_Generation", "Review_Evaluation")
graph.add_edge("Review_Evaluation", END)
# compile the graph
workflow = graph.compile()

In [30]:
initial_state = {"item": "Galaxy S25 Ultra"}
final_state = workflow.invoke(initial_state)

In [31]:
print(final_state["review"])

"I am extremely disappointed with the Galaxy S25 Ultra. The battery life is abysmal, lasting only a few hours before needing to be charged again. The camera quality is also subpar, producing grainy and blurry photos. Additionally, the phone constantly freezes and lags, making it incredibly frustrating to use. Do not waste your money on this disappointment of a phone."


In [32]:
print(final_state["evaluation"])

We are sorry to hear that you are disappointed with your Galaxy S25 Ultra. We strive to provide the best experience for our customers and we apologize if the phone did not meet your expectations. 

We recommend considering other options that may better suit your needs, such as the latest models from different brands or seeking assistance from our customer service team for troubleshooting and potential solutions to the issues you are experiencing. Thank you for sharing your feedback with us.
